In [1]:
import cv2
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
def drawRectangle(frame, bbox):
    point1 = (int(bbox[0]), int(bbox[1]))
    point2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
    cv.rectangle(frame, point1, point2, color=(255,0,0), thickness=2)
    cv.rectangle(frame, point1, point2, (255, 0, 0), 2, 1)
    
def displayRectangle(frame, bbox):
    plt.figure(figsize=(20, 10))
    drawRectangle(frame, bbox)
    frame_copy = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
    plt.imshow(frame)
    plt.axis("off")
    
def draw_fingertip_boxes(frame, fingertip_points):
    for idx, point in enumerate(fingertip_points):
        x, y = point
        cv2.rectangle(frame, (x+40, y+40), (x, y), (0, 255, 0), 2)
        cv2.putText(frame, f"{idx+1}", (x-15, y-15), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

In [3]:
def get_fingertips(contour, detections):
    fingertips = []
    if detections is not None:
        for i in range(detections.shape[0]):
            s,e,f,d = detections[i,0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])
            
            a = np.linalg.norm(np.array(end) - np.array(far))
            b = np.linalg.norm(np.array(start) - np.array(far))
            c = np.linalg.norm(np.array(start) - np.array(end))
            
            angle = np.arccos((b**2 + a**2 - c**2)/(2*b*a + 1e-5))

            if angle <= np.pi / 2 and d > 10000:
                fingertips.append(start)
                fingertips.append(end)
                
    # Remove dups
    unique_fingertips = []
    for pt in fingertips:
        if all(np.linalg.norm(np.array(pt) - np.array(p)) > 22 for p in unique_fingertips):
            unique_fingertips.append(pt)
    return unique_fingertips

In [4]:
cap = cv2.VideoCapture(0)

while True:
    ok, frame = cap.read()
    if not ok:
        break

    frame = cv2.flip(frame, 1)
    roi = frame[100:400, 100:400]
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)

    # Skin color range for HSV
    lower_skin = np.array([0, 30, 60], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)

    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    mask = cv2.GaussianBlur(mask, (5,5), 0)
    mask = cv2.dilate(mask, None, iterations=2)

    contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        max_contour = max(contours, key=cv2.contourArea)
        if cv2.contourArea(max_contour) > 5000:
            hull = cv2.convexHull(max_contour)
            hull_indices = cv2.convexHull(max_contour, returnPoints=False)
            defects = cv2.convexityDefects(max_contour, hull_indices)

            fingertips = get_fingertips(max_contour, defects)
            for pt in fingertips:
                pt_global = (pt[0] + 100, pt[1] + 100)  # Offset by ROI position
                cv2.circle(frame, pt_global, 5, (255,0,0), -1)

            draw_fingertip_boxes(frame, [(pt[0] + 100, pt[1] + 100) for pt in fingertips])

    # Draw ROI
    cv2.rectangle(frame, (0, 0), (1000, 1000), (0, 255, 255), 2)
    cv2.imshow("Fingertip Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

KeyboardInterrupt: 